## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-13-11-48-07 +0000,nyt,Nepal’s New Government Calls Elections. Its Cr...,https://www.nytimes.com/2025/09/13/world/asia/...
1,2025-09-13-11-00-00 +0000,latimes,3 fatal overdoses in L.A. County linked to syn...,https://www.latimes.com/california/story/2025-...
2,2025-09-13-10-54-18 +0000,nyt,The Perilous Path to Escape Gaza City,https://www.nytimes.com/2025/09/13/world/middl...
3,2025-09-13-10-53-41 +0000,bbc,Donald Trump's UK state visit is next week - t...,https://www.bbc.com/news/articles/c8rynvlm6zjo...
4,2025-09-13-10-20-34 +0000,nyt,Zinc Roofs Give Paris Its Signature Look. But ...,https://www.nytimes.com/2025/09/13/zinc-roofs-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2354/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
84,kirk,52
83,charlie,46
25,trump,25
418,shooting,15
1,new,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
75,2025-09-13-02-26-24 +0000,wapo,Charlie Kirk shooting suspect in custody; fami...,https://www.washingtonpost.com/nation/2025/09/...,151
36,2025-09-13-09-00-00 +0000,wsj,"Spencer Cox, Utah’s Republican governor, addre...",https://www.wsj.com/politics/policy/utah-spenc...,147
102,2025-09-13-00-01-58 +0000,nyt,Trump Blames the Left and Downplays ‘Radical R...,https://www.nytimes.com/2025/09/12/us/politics...,147
122,2025-09-12-22-00-15 +0000,nypost,Utah Gov. Spencer Cox tells people to ‘touch g...,https://nypost.com/2025/09/12/us-news/utah-gov...,147
152,2025-09-12-20-56-19 +0000,bbc,"Who is Tyler Robinson, the suspect in custody ...",https://www.bbc.com/news/articles/cp8wl2y66p9o...,146


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
75,151,2025-09-13-02-26-24 +0000,wapo,Charlie Kirk shooting suspect in custody; fami...,https://www.washingtonpost.com/nation/2025/09/...
78,61,2025-09-13-02-07-47 +0000,nypost,Missouri passes Trump-backed redistricting pla...,https://nypost.com/2025/09/12/us-news/missouri...
36,42,2025-09-13-09-00-00 +0000,wsj,"Spencer Cox, Utah’s Republican governor, addre...",https://www.wsj.com/politics/policy/utah-spenc...
94,42,2025-09-13-00-40-07 +0000,nypost,Charlie Kirk’s wife Erika delivers emotional f...,https://nypost.com/2025/09/12/us-news/charlie-...
142,39,2025-09-12-21-08-55 +0000,nypost,Ukraine strikes key Russian oil terminal in la...,https://nypost.com/2025/09/12/world-news/ukrai...
15,37,2025-09-13-10-00-00 +0000,latimes,Urgent calls to end toxic social media spiral:...,https://www.latimes.com/california/story/2025-...
236,35,2025-09-12-15-18-57 +0000,startribune,Another shocking act of political violence lea...,https://www.startribune.com/charlie-kirk-minne...
269,34,2025-09-12-12-50-11 +0000,nypost,New Joni Ernst bill would deactivate governmen...,https://nypost.com/2025/09/12/us-news/new-joni...
183,33,2025-09-12-18-49-00 +0000,wsj,Trump Says He Will Send National Guard to Memphis,https://www.wsj.com/us-news/trump-says-he-will...
144,31,2025-09-12-21-08-15 +0000,cbc,"Protests outside, silence inside as Canada goe...",https://www.cbc.ca/sports/tennis/canada-israel...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
